**Part 3**

* Adding vectorized BlackJack Environment
* Adding Device Agnostic code (GPU Training)
* Attempting to use Softmax (Categorical Distribution) implementation instead of Sigmoid (Binary Bernoulli Distribution)

**Results**

* Still very slow during training

# Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import numpy as np
import gymnasium as gym

# Testing

In [2]:
env = gym.make("Blackjack-v1", sab=True) # `render_mode="human"` creates a pygame popup window to analyze play # `sab=True` uses the Sutton & Barto version

# Agent

In [3]:
class BlackJackAgent(nn.Module):
    def __init__(self, obs_size=3, hidden_size=10, output_size=2):
        super(BlackJackAgent, self).__init__()
        self.layer_1 = nn.Linear(obs_size, hidden_size)
        self.layer_2 = nn.Linear(hidden_size, output_size)
        self.action_probs_activation_layer = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = torch.relu(self.layer_1(x))
        logits = self.layer_2(x)
        return logits       # later use nn.Softmax to get probabilities

    def get_action_probs(self, logits):
        """Get the probabilities of each action."""
        return self.action_probs_activation_layer(logits)
    
    def sample_action(self, action:None):
        """Get the probability of choosing the action"""
        logits = self.forward(action)
        probs = self.get_action_probs(logits)
        dist = torch.distributions.Categorical(probs=probs)
        action = dist.sample().item()
        prob_of_action = dist.log_prob(action)
        return action, prob_of_action

# Training Loop

In [4]:
def training_blackjack_agent(epochs=50, learning_rate=0.0001, batch_size=64, gamma=0.99, k_epochs=64, epsilon=0.2, beta_kl=0.01, max_grad_norm=0.5, entropy_coeff=0.01, log_iterations=10, device="cpu", num_envs=16) -> BlackJackAgent: 
    print(f"Training BlackJack Agent's Policy on {device} with {epochs} epochs, {learning_rate} learning rate, batch size {batch_size}, and KL beta {beta_kl}.")

    vec_env = gym.make_vec("Blackjack-v1", num_envs=num_envs, sab=True) # `sab=True` uses the Sutton & Barto version

    # steps_per_env_per_rollout = batch_size // num_envs if batch_size % num_envs == 0 else (batch_size // num_envs) + 1

    New_Policy = BlackJackAgent().to(device)   # STEP 3 || 
    optimizer = optim.Adam(params=New_Policy.parameters(), lr=learning_rate)


    for epoch in tqdm(range(epochs), desc=f"Main Epoch (Outer Loop)", leave=False):     # STEP 4 || 
        # STEP 5 || Sample a batch D_b from D --> OMITTED 
        # STEP 6 || Update the old policy model PI old <- PI new
        Policy_Old = BlackJackAgent().to(device)
        Policy_Old.load_state_dict(New_Policy.state_dict())
        Policy_Old.eval()   # Prevent Gradient tracking

        # This will store trajectories for all episodes collected in the current batch
        completed_batch_trajectories = []

        # Reset all vectorized environments
        raw_observations, infos = vec_env.reset() # observations is a numpy array of shape (num_envs, obs_dim(3))
        observations = np.stack(raw_observations, axis=1)
        dones = np.array([False] * num_envs) # Track the done status for each parallel environment
        truncateds = np.array([False] * num_envs) # Track truncated status for each parallel environment

        # Initialize current trajectories for all parallel environments
        # Each element in this list will be a dict for an *in-progress* episode in a specific env
        current_episode_trajectories = [{"states": [], "actions": [], "rewards": [], "log_probs": []} for _ in range(num_envs)]

        # --- STEP 7 Collect a Batch of Experiences Using the Old Policy---
        # Loop Agent prediction, recording trajectories to lists:
        episodes_collected_in_batch = 0
        max_steps_per_batch_limit = batch_size * 5 # A safety limit to prevent infinite loops if episodes are very long
        current_total_steps = 0

        while episodes_collected_in_batch < batch_size and current_total_steps < max_steps_per_batch_limit:
            obs_tensor = torch.tensor(observations, dtype=torch.float32).to(device)

            with torch.no_grad():
                logits = Policy_Old(obs_tensor)
                dist = torch.distributions.Categorical(logits=logits)
                actions = dist.sample() # Tensor of shape [1]
                log_probs = dist.log_prob(actions)
                    
            raw_next_obs, rewards, dones, truncateds, infos = vec_env.step(actions.cpu().numpy()) # actions must be on CPU for env.step()
            next_obs = np.stack(raw_next_obs, axis=1)
            current_total_steps += num_envs

            # Process data for each parallel environment
            for env_idx in range(num_envs):
                
                obs_to_append = observations[env_idx]
                if isinstance(obs_to_append, torch.Tensor):
                    obs_to_append = obs_to_append.cpu().numpy()
                # Store current_episode_trajectories
                current_episode_trajectories[env_idx]["states"].append(obs_to_append)
                current_episode_trajectories[env_idx]["actions"].append(actions[env_idx].item())
                current_episode_trajectories[env_idx]["rewards"].append(rewards[env_idx])
                current_episode_trajectories[env_idx]["log_probs"].append(log_probs[env_idx].cpu())
                
                if dones[env_idx] or truncateds[env_idx]:
                    completed_batch_trajectories.append(current_episode_trajectories[env_idx])
                    episodes_collected_in_batch += 1

                    # Reset this specific environment
                    # new_obs, new_info = vec_env.reset_at(env_idx)
                    # observations[env_idx] = new_obs

                    current_episode_trajectories[env_idx] = {"states": [], "actions": [], "rewards": [], "log_probs": []}

            observations = next_obs  # Update the observation

        for env_idx in range(num_envs):
            if len(current_episode_trajectories[env_idx]["states"]) > 0:
                # If there's partial data, it means the episode was still running
                # when `batch_size` was met. You'll need to decide how to handle this.
                # For simplicity for now, we'll append them. In full PPO, you'd add
                # the value of the last state to its rewards.
                completed_batch_trajectories.append(current_episode_trajectories[env_idx])
                # Note: These might not be "full" episodes in the sense of reaching a done state,
                # but they contribute steps to your batch.

        # These lists will hold data from ALL episodes in the current batch for Advantage Calculation
        all_states = []
        all_actions = []
        all_old_log_probs = []
        all_discounted_rewards = []

        # STEP 8 || Calculate Discounted Rewards for completed trajectories
        for episode_trajectory in completed_batch_trajectories: 
            rewards = episode_trajectory["rewards"]
            states = episode_trajectory["states"]
            actions = episode_trajectory["actions"]
            log_probs = episode_trajectory["log_probs"]
            
            if not rewards:
                continue

            discounted_reward = 0
            returns_for_episode = []
            for reward in reversed(rewards):
                discounted_reward = reward + gamma * discounted_reward
                returns_for_episode.insert(0, discounted_reward)

            discounted_rewards = torch.tensor(returns_for_episode, dtype=torch.float32)
            # print(f"discounted_rewards size: {discounted_rewards.size()}")
            # Add each trajectory information for the batch
            if states:
                all_states.extend(states)
                all_actions.extend(actions)
                all_old_log_probs.extend(log_probs)
                all_discounted_rewards.extend(discounted_rewards.tolist())

        # --- IMPORTANT: Pre-tensorization checks and conversions ---
        if not all_states or not all_actions or not all_old_log_probs or not all_discounted_rewards:
            print(f"Warning: Epoch {epoch + 1}: Insufficient data collected for optimization. "
                  f"Skipping policy update for this epoch.")
            print(f"  Counts: States={len(all_states)}, Actions={len(all_actions)}, "
                  f"LogProbs={len(all_old_log_probs)}, Rewards={len(all_discounted_rewards)}")
            continue
        # Convert all collected batch data into PyTorch tensors
        all_states_tensor = torch.tensor(np.array(all_states), dtype=torch.float32).to(device)
        all_actions_tensor = torch.tensor(all_actions, dtype=torch.long).to(device)
        # Stack individual log_prob tensors and then flatten if necessary
        all_old_log_probs_tensor = torch.tensor(all_old_log_probs, dtype=torch.float32).to(device) # Ensure it's a 1D tensor
        all_discounted_rewards_tensor = torch.tensor(all_discounted_rewards, dtype=torch.float32).to(device)

        # STEP 9 || Calculate the Advantage of each Time Step for each Trajectory using normalization
        all_advantages_tensor = (all_discounted_rewards_tensor - all_discounted_rewards_tensor.mean()) / (all_discounted_rewards_tensor.std() + 1e-8)

        # Detach these tensors from any computation graph history
        # as they represent fixed data for the policy updates in k_epochs.
        # This prevents the "RuntimeError: Trying to backward through the graph a second time".
        all_states_tensor = all_states_tensor.detach()
        all_actions_tensor = all_actions_tensor.detach()
        all_old_log_probs_tensor = all_old_log_probs_tensor.detach()
        all_advantages_tensor = all_advantages_tensor.detach()

        New_Policy.train()  # Prepare NN for updates

        # --- STEP 10 || GRPO Optimization ---
        for k_epoch in tqdm(range(k_epochs), desc=f"Epoch {epoch+1}/{epochs} (Inner K-Epochs)", leave=True):
            new_logits = New_Policy(all_states_tensor)
            new_dist = torch.distributions.Categorical(logits=new_logits)
            new_log_probs = new_dist.log_prob(all_actions_tensor)
            entropy = new_dist.entropy().mean() # Calculate entropy for regularization

            R1_ratio = torch.exp(new_log_probs - all_old_log_probs_tensor)

            unclipped_surrogate = R1_ratio * all_advantages_tensor
            clipped_surrogate = torch.clamp(input=R1_ratio, min=1.0-epsilon, max=1.0+epsilon) * all_advantages_tensor

            policy_loss = -torch.min(unclipped_surrogate, clipped_surrogate).mean()

            # --- KL Divergence Calculation ---
            # Create distributions for old policies using the trajectory states
            with torch.no_grad():
                old_logits = Policy_Old(all_states_tensor)
            old_dist = torch.distributions.Categorical(logits=old_logits)

            # Calculate KL divergence per sample, then take the mean over the batch
            kl_div_per_sample = torch.distributions.kl.kl_divergence(p=new_dist, q=old_dist)
            kl_loss = kl_div_per_sample.mean() # Mean over the batch

            # Total Loss for GRPO
            total_loss = policy_loss + beta_kl * kl_loss - entropy_coeff * entropy

            # STEP 11 || Policy Updates
            optimizer.zero_grad()
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(New_Policy.parameters(), max_grad_norm)
            optimizer.step()    # Update policy parameters using gradient ascent
        
        
        # --- 4. Logging and Evaluation ---
        if (epoch + 1) % log_iterations == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss.item():.4f}, Ratio: {R1_ratio.mean().item():.5f}, Entropy Term: {entropy:.5f}")
            # You can add more evaluation metrics here, e.g., average reward per episode
            # For Blackjack, the reward is often -1, 0, or 1.
            avg_reward = sum(sum(ep["rewards"]) for ep in completed_batch_trajectories) / len(completed_batch_trajectories) if len(completed_batch_trajectories) > 0 else 0
            print(f"Average reward per episode in batch: {avg_reward:.2f}")

    New_Policy.eval()   # Change to eval mode for evaluation


    vec_env.close() # Close the environment after training
    print("Training complete.")
    return New_Policy # Return the trained policy

In [ ]:
_ = training_blackjack_agent(epochs=50, learning_rate=0.0001, batch_size=64, gamma=0.99, k_epochs=64, epsilon=0.2, beta_kl=0.01, max_grad_norm=0.5, entropy_coeff=0.01, log_iterations=10, device="cpu", num_envs=16)

Training BlackJack Agent's Policy on cpu with 50 epochs, 0.0001 learning rate, batch size 64, and KL beta 0.01.


Main Epoch (Outer Loop):  20%|██        | 10/50 [00:02<00:09,  4.19it/s]

Epoch 10/50, Loss: -0.0254, Ratio: 0.99264, Entropy Term: 0.62366
Average reward per episode in batch: -0.22


Main Epoch (Outer Loop):  40%|████      | 20/50 [00:04<00:05,  5.94it/s]

Epoch 20/50, Loss: -0.0123, Ratio: 0.99465, Entropy Term: 0.35454
Average reward per episode in batch: -0.04


...

Epoch 30/50, Loss: -0.0095, Ratio: 1.00775, Entropy Term: 0.35954
Average reward per episode in batch: -0.16


Main Epoch (Outer Loop):  80%|████████  | 40/50 [00:08<00:02,  3.72it/s]

Epoch 40/50, Loss: -0.0116, Ratio: 0.98755, Entropy Term: 0.39186
Average reward per episode in batch: 0.00


Epoch 50/50 (Inner K-Epochs): 100%|██████████| 64/64 [00:00<00:00, 536.35it/s]
                                                                        

Epoch 50/50, Loss: -0.0065, Ratio: 1.00064, Entropy Term: 0.28929
Average reward per episode in batch: 0.00
Training complete.


Training BlackJack Agent's Policy on cpu with 50 epochs, 0.0001 learning rate, batch size 64, and KL beta 0.01.
                                                               
(array([21, 12, 19, 19, 20, 15, 18, 16, 10,  7, 19, 10, 17, 15, 14,  9]), array([ 5,  5,  1,  4, 10, 10,  3,  9,  2,  6,  1,  6,  9,  5,  9,  8]), array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))


In [ ]:
# _ = training_blackjack_agent(epochs=50, learning_rate=0.0001, batch_size=64, gamma=0.99, k_epochs=64, epsilon=0.2, beta_kl=0.01, max_grad_norm=0.5, entropy_coeff=0.01, log_iterations=10, device="cuda", num_envs=16)

Training BlackJack Agent's Policy on cpu with 50 epochs, 0.0001 learning rate, batch size 64, and KL beta 0.01.


Main Epoch (Outer Loop):  20%|██        | 10/50 [00:01<00:05,  6.79it/s]

Epoch 10/50, Loss: -0.0003, Ratio: 1.00001, Entropy Term: 0.01753
Average reward per episode in batch: -0.86


Main Epoch (Outer Loop):  40%|████      | 20/50 [00:03<00:05,  5.40it/s]

Epoch 20/50, Loss: -0.0013, Ratio: 1.00067, Entropy Term: 0.01085
Average reward per episode in batch: -0.86


...

Epoch 30/50, Loss: -0.0002, Ratio: 0.99980, Entropy Term: 0.01344
Average reward per episode in batch: -0.86


Main Epoch (Outer Loop):  80%|████████  | 40/50 [00:07<00:01,  5.87it/s]

Epoch 40/50, Loss: -0.0004, Ratio: 0.99995, Entropy Term: 0.03450
Average reward per episode in batch: -0.86


Epoch 50/50 (Inner K-Epochs): 100%|██████████| 64/64 [00:00<00:00, 425.77it/s]
                                                                        

Epoch 50/50, Loss: -0.0005, Ratio: 0.99904, Entropy Term: 0.03725
Average reward per episode in batch: -0.91
Training complete.


Training BlackJack Agent's Policy with 10 epochs, 0.0001 learning rate, batch size 4, and KL beta 0.01.
* Batch of Trajectories:
* [{'states': [(12, 10, 0)], 'actions': [0], 'rewards': [-1.0], 'log_probs': [tensor([-0.1239])]}, 
* {'states': [(20, 7, 0)], 'actions': [0], 'rewards': [1.0], 'log_probs': [tensor([-0.0815])]}, 
* {'states': [(12, 1, 0), (17, 1, 0)], 'actions': [1, 1], 'rewards': [0.0, -1.0], 'log_probs': [tensor([-1.5968]), tensor([-1.9474])]}, 
* {'states': [(6, 6, 0)], 'actions': [0], 'rewards': [-1.0], 'log_probs': [tensor([-0.2144])]}, 
* {'states': [(7, 4, 0)], 'actions': [0], 'rewards': [-1.0], 'log_probs': [tensor([-0.2734])]}, 
* {'states': [(13, 3, 1)], 'actions': [0], 'rewards': [-1.0], 'log_probs': [tensor([-0.1471])]}, 
* {'states': [(15, 10, 0)], 'actions': [0], 'rewards': [-1.0], 'log_probs': [tensor([-0.1000])]}, 
* {'states': [(12, 10, 0)], 'actions': [0], 'rewards': [1.0], 'log_probs': [tensor([-0.1239])]}, 
* {'states': [(14, 7, 0)], 'actions': [0], 'rewards': [-1.0], 'log_probs': [tensor([-0.1320])]}]

In [7]:
# Example usage (assuming you have a way to call this function, e.g., in a main block)
if __name__ == '__main__':
    # You can adjust these parameters as needed
    # Using a larger batch_size for more stable training and to reduce empty batch issues
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   # Device Agnostic Code
    trained_policy = training_blackjack_agent(
        epochs=2000,
        learning_rate=0.0003,
        batch_size=2048, # Significantly larger batch size recommended for stability
        k_epochs=128,
        epsilon=0.2,
        beta_kl=0.01,
        entropy_coeff=0.001,
        log_iterations=100,
        gamma=0.99,
        device=device,
        num_envs=16
    )

    print("\nTesting the trained policy:")
    test_env = gym.make("Blackjack-v1", sab=True)
    total_test_rewards = 0
    num_test_episodes = 1000

    for _ in range(num_test_episodes):
        obs, _ = test_env.reset()
        done = False
        truncated = False
        episode_reward = 0
        while not done and not truncated:
            obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0).to(device)
            with torch.no_grad():
                logits = trained_policy(obs_tensor)
                dist = torch.distributions.Categorical(logits=logits)
                action = dist.sample()
            obs, reward, done, truncated, _ = test_env.step(action.item())
            episode_reward += reward
        total_test_rewards += episode_reward

    print(f"Average reward over {num_test_episodes} test episodes: {total_test_rewards / num_test_episodes:.4f}")
    test_env.close()

Training BlackJack Agent's Policy on cpu with 2000 epochs, 0.0003 learning rate, batch size 2048, and KL beta 0.01.


Main Epoch (Outer Loop):   5%|▌         | 100/2000 [02:18<42:00,  1.33s/it]

Epoch 100/2000, Loss: -0.0015, Ratio: 0.99912, Entropy Term: 0.21172
Average reward per episode in batch: -0.02


Main Epoch (Outer Loop):  10%|█         | 200/2000 [05:01<1:27:11,  2.91s/it]

Epoch 200/2000, Loss: -0.0014, Ratio: 0.99874, Entropy Term: 0.17166
Average reward per episode in batch: -0.06


...

Epoch 1800/2000, Loss: -0.0005, Ratio: 0.99909, Entropy Term: 0.06964
Average reward per episode in batch: -0.02


Main Epoch (Outer Loop):  95%|█████████▌| 1900/2000 [56:46<03:34,  2.15s/it]

Epoch 1900/2000, Loss: -0.0012, Ratio: 0.99973, Entropy Term: 0.08203
Average reward per episode in batch: -0.07


Epoch 2000/2000 (Inner K-Epochs): 100%|██████████| 128/128 [00:01<00:00, 105.11it/s]


Epoch 2000/2000, Loss: -0.0015, Ratio: 0.99895, Entropy Term: 0.09015
Average reward per episode in batch: -0.08
Training complete.

Testing the trained policy:
Average reward over 1000 test episodes: -0.1140


took 32 minutes to run using the CPU

Parameters: 


epochs=2000,
        learning_rate=0.0003,
        batch_size=2048, # Significantly larger batch size recommended for stability
        k_epochs=128,
        epsilon=0.2,
        beta_kl=0.01,
        entropy_coeff=0.001,
        log_iterations=100,
        gamma=0.99

In [8]:
test_env = gym.make("Blackjack-v1", render_mode="rgb", sab=True)
total_test_rewards = 0

c:\Users\brianperez\AppData\Local\anaconda3\envs\GRPO_env\lib\site-packages\gymnasium\envs\registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='rgb' that is not in the possible render_modes (['human', 'rgb_array']).
  logger.warn(


In [9]:
num_test_episodes = 10

In [10]:
for episode in range(num_test_episodes):
    print(f"Resetting env for episode: {episode}")
    obs, _ = test_env.reset()
    done = False
    truncated = False
    episode_reward = 0
    stored_obs=[]
    while not done and not truncated:
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            logits = trained_policy(obs_tensor)
            dist = torch.distributions.Categorical(logits=logits)
            action = dist.sample()
            print(f"obs_tensor: {obs_tensor} || Action taken: {action}")
        obs, reward, done, truncated, _ = test_env.step(action.item())
        episode_reward += reward
        if (truncated): print("truncated")
    print(f"Reward: {episode_reward} || Final Observation: {obs}")

Resetting env for episode: 0
obs_tensor: tensor([[16., 10.,  0.]]) || Action taken: tensor([0])
Reward: -1.0 || Final Observation: (16, 10, 0)
Resetting env for episode: 1
obs_tensor: tensor([[10.,  2.,  0.]]) || Action taken: tensor([1])
obs_tensor: tensor([[13.,  2.,  0.]]) || Action taken: tensor([1])
Reward: -1.0 || Final Observation: (23, 2, 0)
Resetting env for episode: 2
obs_tensor: tensor([[ 6., 10.,  0.]]) || Action taken: tensor([1])
obs_tensor: tensor([[ 9., 10.,  0.]]) || Action taken: tensor([1])
obs_tensor: tensor([[11., 10.,  0.]]) || Action taken: tensor([1])
obs_tensor: tensor([[13., 10.,  0.]]) || Action taken: tensor([1])
Reward: -1.0 || Final Observation: (22, 10, 0)
Resetting env for episode: 3
obs_tensor: tensor([[21.,  7.,  1.]]) || Action taken: tensor([0])
Reward: 1.0 || Final Observation: (21, 7, 1)
Resetting env for episode: 4
obs_tensor: tensor([[19., 10.,  0.]]) || Action taken: tensor([0])
Reward: 1.0 || Final Observation: (19, 10, 0)
Resetting env for epi

In [11]:
env.close()

Currently the final state which reveals what the dealer ended up with in the end is not shown. By trying to access the dealer's final hand or by adding custom logging within the environment, you'll gain the critical information needed to definitively understand the why behind each reward.

In [13]:
SAVE_LOCATION = "app/model_weights/blackjack_policy_model.pth"

torch.save(trained_policy.state_dict(), f=SAVE_LOCATION)